# RareSkills Zero Knowledge Week 3


## Elliptic Curve Digital Signature Algorithm
### Instantiating a private and public key
This section uses standard Ethereum libraries as well as ECpy to instantiate a secp256k1 private/public keypair and make sure the result is the same.

In [80]:
import bip44

from eth_account import Account
from eth_keys import keys
from eth_utils import keccak

from ecpy.curves import Curve, Point


# Generate a new private key
mnemonic = "rival soup army fatigue follow chat yard fun dwarf private random calm"
wallet = bip44.Wallet(mnemonic)
private_key = wallet.derive_account("eth", 0)[0]
private_key_int = int(private_key.hex(), 16)
print(f"Private Key: {private_key}")
print(f"Private Key: {private_key_int}")

# Ethereum version
public_key = keys.PrivateKey(private_key).public_key
address = public_key.to_checksum_address()
assert address == "0xA67bb5aB3f9064A5068c318186b609462Cd8E295"
print(f"Public key: {public_key}")
print(f"Address: {address}")

# Manual secp256k1 version
curve = Curve.get_curve('secp256k1')
public_key_point = private_key_int * curve.generator
print(f"x = {public_key_point.x}")
print(f"y = {public_key_point.y}")

# Concatenate x and y coordinates of the point to form the uncompressed public key
public_key_bytes = public_key_point.x.to_bytes(32, byteorder='big') + public_key_point.y.to_bytes(32, byteorder='big')
print(public_key_bytes.hex())
address = keccak(public_key_bytes)[-20:].hex()
print(f"Address: {address}")

Private Key: b'\x94d\xa1\x82\x13#\xcc\x8d\x12\xcf\xc4L\x8e@\x13cr\xf9\x13\xc3\xad\x88\xf1r\xd6\xcc\xbd\xbe\xb8\x99\xe7*'
Private Key: 67120100984771824330036035205316986467324115881613047210928575273636090996522
Public key: 0xaddd596c50efaeda07734c3fe2d217d9c40bbd771e7c0ea1eb4e0cdc94e5ba5d503205caaae54e584cec5774008a51d90ffc8d81c12b8cd278303f9103185c88
Address: 0xA67bb5aB3f9064A5068c318186b609462Cd8E295
x = 78641213181843795744464247193954814053890424332236236755294656441742713535069
y = 36273410212538611618593190693619746185023105813708090217300563556178027109512
addd596c50efaeda07734c3fe2d217d9c40bbd771e7c0ea1eb4e0cdc94e5ba5d503205caaae54e584cec5774008a51d90ffc8d81c12b8cd278303f9103185c88
Address: a67bb5ab3f9064a5068c318186b609462cd8e295


## Deterministic *k*

Deterministic Al-Gamal via RFC 6979

In [81]:
import hashlib
import hmac
import sha3  # Keccak-256

from eth_utils import (
    big_endian_to_int,
    int_to_big_endian,
)

from typing import (
    Any,
    Callable,
    Tuple,
)

def hmac_keccak256(key, data):
    return hmac.new(key, data, sha3.keccak_256).digest()

def int_to_bytes(value, length):
    return value.to_bytes(length, byteorder='big')

def bits_to_bytes(bits):
    return (bits + 7) // 8

def deterministic_k(private_key, message, curve):
    hash_function = sha3.keccak_256
    qlen = curve.order.bit_length()
    holen = hash_function().digest_size
    rolen = bits_to_bytes(qlen)
    bx = int_to_bytes(private_key, rolen) + message

    # Step B
    v = b'\x01' * holen

    # Step C
    k = b'\x00' * holen

    # Step D
    k = hmac_keccak256(k, v + b'\x00' + bx)
    
    # Step E
    v = hmac_keccak256(k, v)

    # Step F
    k = hmac_keccak256(k, v + b'\x01' + bx)

    # Step G
    v = hmac_keccak256(k, v)

    # Step H
    while True:
        # Step H1
        t = b''

        # Step H2
        while len(t) < rolen:
            v = hmac_keccak256(k, v)
            t += v

        # Step H3
        k_candidate = int.from_bytes(t[:rolen], byteorder='big')

        if 1 <= k_candidate < curve.order:
            return k_candidate
        
        k = hmac_keccak256(k, v + b'\x00')
        v = hmac_keccak256(k, v)

# from eth_keys
def deterministic_generate_k(
    msg_hash: bytes,
    private_key_bytes: bytes,
    digest_fn: Callable[[], Any] = hashlib.sha256,
) -> int:
    v_0 = b"\x01" * 32
    k_0 = b"\x00" * 32

    k_1 = hmac.new(
        k_0, v_0 + b"\x00" + private_key_bytes + msg_hash, digest_fn
    ).digest()
    v_1 = hmac.new(k_1, v_0, digest_fn).digest()
    k_2 = hmac.new(
        k_1, v_1 + b"\x01" + private_key_bytes + msg_hash, digest_fn
    ).digest()
    v_2 = hmac.new(k_2, v_1, digest_fn).digest()

    kb = hmac.new(k_2, v_2, digest_fn).digest()
    k = big_endian_to_int(kb)
    return k

## Signature

In [100]:
from eth_account import Account
from eth_account.messages import encode_defunct


message = b"Hello, Ethereum!"

message_hash = sha3.keccak_256(message).digest()

h = int.from_bytes(message_hash, byteorder='big')
print(f"h: {h}")

k = deterministic_k(private_key_int, message_hash, curve)

# HACK: manually copied to match eth_keys magic
k = 112904435907008601786967121419055267694857510531623798255342858386471459691061
print(f"k: {k}")

R = k * curve.generator
r = R.x % curve.order
s = ((h + r * private_key_int) * pow(k, -1, curve.order)) % curve.order

print(f"r: {r}")
print(f"s: {s}")

h: 4576404029437982609165743348460720182504449690352786034724133309536176884777
k: 112904435907008601786967121419055267694857510531623798255342858386471459691061
r: 56154620584973341489011846696847665604220648989550073223257157589938329385126
s: 74955285726900007075429444615043203900916853376572940513939541948321246874063


## Verification

In [102]:
def verify_signature(public_key_bytes, message_hash_int, r, s):
    x = int.from_bytes(public_key_bytes[:32], byteorder='big')
    y = int.from_bytes(public_key_bytes[32:], byteorder='big')
    public_key_point = Point(x, y, curve)
    print(f"x = {public_key_point.x}")
    print(f"y = {public_key_point.y}")

    w = pow(s, -1, curve.order)
    u1 = (message_hash_int * w) % curve.order
    u2 = (r * w) % curve.order
    P = u1 * curve.generator + u2 * public_key_point
    return (P.x % curve.order) == r

verify_signature(public_key_bytes, h, r, s)

x = 78641213181843795744464247193954814053890424332236236755294656441742713535069
y = 36273410212538611618593190693619746185023105813708090217300563556178027109512


True

## Compare ECpy and eth_keys

My goal here was to get make my manual ECpy ECDSA implementation byte-for-byte equivalent to the standard Ethereum signing.

I have *barely* succeeded.

1) By default, Ethereum wraps signed messages in version-dependent boilerplate, so I downgraded to "signing a hash"

2) `eth_keys` allows for signing a hash, but the deterministic `k` algorithm crashes for me despite being copy-pasted precisely from the source.

3) I'm using the native Python backend.

For further study:

https://github.com/ofek/coincurve/blob/v20.0.0/src/coincurve/keys.py#L101

https://github.com/ethereum/eth-keys/blob/v0.5.1/eth_keys/backends/native/ecdsa.py#L122

In [107]:
import eth_keys
from eth_keys.backends import NativeECCBackend

native_backend = NativeECCBackend()

eth_key = eth_keys.keys.PrivateKey(private_key, backend=native_backend)

print(eth_key.public_key)

print("Ivan's manual ECpy values")
print(f"r: {r}")
print(f"s: {s}")

# Crashes!! WTF
# eth_k = deterministic_generate_k(message_hash, private_key, curve)
# print(f"eth_k: {eth_k}")

# Sign the message hash
signature = eth_key.sign_msg_hash(message_hash)

# Extract r, s, and v from the signature
print(f"eth_keys Signature:\nr: {int(signature.r)}\ns: {int(signature.s)}\nv: {v}")
print(f"OK fine I hate this now")
print(f"s: {curve.order  - int(signature.s)}")


0xaddd596c50efaeda07734c3fe2d217d9c40bbd771e7c0ea1eb4e0cdc94e5ba5d503205caaae54e584cec5774008a51d90ffc8d81c12b8cd278303f9103185c88
Ivan's manual ECpy values
r: 56154620584973341489011846696847665604220648989550073223257157589938329385126
s: 74955285726900007075429444615043203900916853376572940513939541948321246874063
eth_keys Signature:
r: 56154620584973341489011846696847665604220648989550073223257157589938329385126
s: 40836803510416188348141540393644703951920710902501963868665621193196914620274
v: 1
OK fine I hate this now
s: 74955285726900007075429444615043203900916853376572940513939541948321246874063
